In [1]:
# Libraries
import datetime
import os
import sqlite3
import pandas as pd
import os
from platform import python_version
from datetime import datetime

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

c:\ZHAW\Sc_Programming\Project\Github\sp_project_janosh_marius\backend_sp_db


In [3]:
exportdir = ['Imports', 'Exports', 'Database']


for exportdirs in exportdir:
    output_dir = os.path.join(r'C:\ZHAW\Sc_Programming\Project\Github\sp_project_janosh_marius\backend_sp_db', exportdirs, )

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)



In [4]:
conn = sqlite3.connect("Database/project_SP.db")

In [5]:
conn.execute('''CREATE TABLE IF NOT EXISTS SP_Project
             (ID INT PRIMARY KEY     NOT NULL);''')

In [8]:
# Import data
df_cust = pd.read_excel(r"Imports/Januar_21.xlsx", sheet_name='Test2')




# Write data to the data base table named 'COMPANY'
df_cust.to_sql('SP_Project', conn, if_exists='replace')

# Commit the changes to the table
conn.commit()



# Close connection to db
conn.close()